In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [14]:
data=pd.read_csv('clean_data.csv')
data.head()

C:\Users\andri\AppData\Local\Temp\ipykernel_45356\3132916111.py:1: DtypeWarning: Columns (14,17) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.read_csv('clean_data.csv')


,Origin,Destination,TTT,LOS,Arrival Time,Return Arrival Time,Onward Departure Airport,Onward Arrival Airport,Return Departure Airport,Return Arrival Airport,...,Departure Time Category Num,Return Time Category Num,Is Weekend Flight,Onward Departure Airport Distance,Onward Arrival Airport Distance,Return Departure Airport Distance,Return Arrival Airport Distance,Total Connections,Total Flight Duration(min),Flight Duration Binned
0,London,Paris,1,1,3/7/2025 16:30,3/8/2025 19:25,SEN,CDG,ORY,LGW,...,2,3,True,64,25,13,45,0,135,0-200
1,London,Paris,1,1,3/7/2025 16:30,3/8/2025 14:25,SEN,CDG,ORY,LHR,...,2,2,True,64,25,13,23,0,145,0-200
2,London,Paris,1,1,3/7/2025 16:30,3/8/2025 14:50,SEN,CDG,CDG,LGW,...,2,2,True,64,25,25,45,0,135,0-200
3,London,Paris,1,1,3/7/2025 16:30,3/8/2025 12:00,SEN,CDG,ORY,LGW,...,2,1,True,64,25,13,45,0,140,0-200
4,London,Paris,1,1,3/7/2025 16:30,3/8/2025 15:20,SEN,CDG,ORY,LGW,...,2,2,True,64,25,13,45,0,140,0-200


In [15]:
data.columns

Index(['Origin', 'Destination', 'TTT', 'LOS', 'Arrival Time',
       'Return Arrival Time', 'Onward Departure Airport',
       'Onward Arrival Airport', 'Return Departure Airport',
       'Return Arrival Airport', 'Onward Airlines', 'Return Airlines',
       'Onward Flight Connections', 'Onward Connection Duration(min)',
       'Onward Flight Connection Airport', 'Return Flight Connections',
       'Return Connection Duration(min)', 'Return Flight Connection Airport',
       'Carry-ons', 'Checked Bags', 'Price(NIS)', 'Snapshot Date', 'Site',
       'Total Onward Flight Duration(min)',
       'Total Return Flight Duration(min)', 'Departure Month', 'Departure Day',
       'Departure Weekday', 'Return Month', 'Return Day', 'Return Weekday',
       'Departure Time Category Num', 'Return Time Category Num',
       'Is Weekend Flight', 'Onward Departure Airport Distance',
       'Onward Arrival Airport Distance', 'Return Departure Airport Distance',
       'Return Arrival Airport Distance', 

In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 527232 entries, 0 to 527231
Data columns (total 41 columns):
 #   Column                             Non-Null Count   Dtype 
---  ------                             --------------   ----- 
 0   Origin                             527232 non-null  object
 1   Destination                        527232 non-null  object
 2   TTT                                527232 non-null  int64 
 3   LOS                                527232 non-null  int64 
 4   Arrival Time                       527232 non-null  object
 5   Return Arrival Time                527232 non-null  object
 6   Onward Departure Airport           527232 non-null  object
 7   Onward Arrival Airport             527232 non-null  object
 8   Return Departure Airport           527232 non-null  object
 9   Return Arrival Airport             527232 non-null  object
 10  Onward Airlines                    527232 non-null  object
 11  Return Airlines                    527232 non-null  

## Using Gussian Regression
* We need to choose carefully which features effect the price, because the model uses O^3 memory to run.
* Other approach can be to reduce the rows in the data and  train on a smaller datebase.
* We already calculated feature importance, this we will use the top 5 iomportant features that we was in the XG model. 
* Gussian does't support categorical columns, so we need to conver them to dummies.
* We would want to avoid using Airlines, because one-hot encoding increases dimentions significantly.

## Features:
* Total Onward Flight Duration (we will use the overall duration, because it consists of 2 columns)
* Origin
* Departure Day
* Departure Weekday
* Destination

In [17]:
data = pd.get_dummies(data, columns=['Origin', 'Destination'], drop_first=True)


In [18]:
data.columns

Index(['TTT', 'LOS', 'Arrival Time', 'Return Arrival Time',
       'Onward Departure Airport', 'Onward Arrival Airport',
       'Return Departure Airport', 'Return Arrival Airport', 'Onward Airlines',
       'Return Airlines', 'Onward Flight Connections',
       'Onward Connection Duration(min)', 'Onward Flight Connection Airport',
       'Return Flight Connections', 'Return Connection Duration(min)',
       'Return Flight Connection Airport', 'Carry-ons', 'Checked Bags',
       'Price(NIS)', 'Snapshot Date', 'Site',
       'Total Onward Flight Duration(min)',
       'Total Return Flight Duration(min)', 'Departure Month', 'Departure Day',
       'Departure Weekday', 'Return Month', 'Return Day', 'Return Weekday',
       'Departure Time Category Num', 'Return Time Category Num',
       'Is Weekend Flight', 'Onward Departure Airport Distance',
       'Onward Arrival Airport Distance', 'Return Departure Airport Distance',
       'Return Arrival Airport Distance', 'Total Connections',
    

In [19]:
X= data[['Total Flight Duration(min)','Origin_Paris','Origin_Rome','Destination_Paris','Destination_Rome','Departure Day','Departure Weekday']]
y = data['Price(NIS)']

In [20]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)


In [21]:
# Normalize the data
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer


numeric_features = ['Total Flight Duration(min)', 'Departure Day', 'Departure Weekday']
scaler = ColumnTransformer([
    ('num', StandardScaler(), numeric_features)  # Scale only numeric features
], remainder='passthrough')  # Keep categorical features unchanged

# Fit and transform the data
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

y_scaler = StandardScaler()
y_train_scaled = y_scaler.fit_transform(y_train.values.reshape(-1, 1)).ravel()


In [22]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_error

kernel = C(1.0) * RBF(length_scale=1.0)
gpr = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=5)



## As we can see, the optimistic approach didnt work, so we need alternative approach: 
* Train on much smaller dataset.
* Train in batches.

we will try to implament the first one. 

In [23]:
X_train.describe()

,Total Flight Duration(min),Departure Day,Departure Weekday
count,369062.000000,369062.000000,369062.000000
mean,242.659889,16.343127,3.020457
std,81.719454,9.002788,1.980136
min,130.000000,1.000000,0.000000
25%,155.000000,8.000000,1.000000
50%,255.000000,17.000000,3.000000
75%,310.000000,24.000000,5.000000
max,3635.000000,31.000000,6.000000


In [24]:

# Ensure X_train_scaled is a NumPy array
if not isinstance(X_train_scaled, np.ndarray):
    X_train_scaled = X_train_scaled.to_numpy()
if not isinstance(y_train_scaled, np.ndarray):
    y_train_scaled = y_train_scaled.to_numpy()

# Define sample size
sample_size = min(10000, len(X_train_scaled))

# Generate sequential indices to ensure valid bounds
valid_indices = np.arange(len(X_train_scaled))

np.random.seed(101)

# Randomly sample from valid indices
subset_indices = np.random.choice(valid_indices, sample_size, replace=False)

# Select the subset of data
X_train_subset = X_train_scaled[subset_indices]
y_train_subset = y_train_scaled[subset_indices]

# Fit the GPR model
gpr.fit(X_train_subset, y_train_subset)


c:\ProgramData\anaconda3\Lib\site-packages\sklearn\gaussian_process\kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
c:\ProgramData\anaconda3\Lib\site-packages\sklearn\gaussian_process\kernels.py:442: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(


GaussianProcessRegressor(kernel=1**2 * RBF(length_scale=1),
                         n_restarts_optimizer=5)

In [25]:
y_pred = gpr.predict(X_test_scaled)



mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mse)

print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"R² Score: {r2:.4f}")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")

Mean Squared Error (MSE): 395741.13
R² Score: -8.2206
Mean Absolute Error (MAE): 594.20
Root Mean Squared Error (RMSE): 629.08


In [28]:
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C, ExpSineSquared

kernel = (
    C(1.0) * RBF(length_scale=[1.0]*7, length_scale_bounds=(1e-2, 1e2))  # Captures smooth trends
    + ExpSineSquared(length_scale=1.0, periodicity=7)  # Captures weekly price patterns
)

gpr = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10)

In [ ]:

# Ensure X_train_scaled is a NumPy array
if not isinstance(X_train_scaled, np.ndarray):
    X_train_scaled = X_train_scaled.to_numpy()
if not isinstance(y_train_scaled, np.ndarray):
    y_train_scaled = y_train_scaled.to_numpy()

# Define sample size
sample_size = min(10000, len(X_train_scaled))

# Generate sequential indices to ensure valid bounds
valid_indices = np.arange(len(X_train_scaled))

np.random.seed(101)

# Randomly sample from valid indices
subset_indices = np.random.choice(valid_indices, sample_size, replace=False)

# Select the subset of data
X_train_subset = X_train_scaled[subset_indices]
y_train_subset = y_train_scaled[subset_indices]

# Fit the GPR model
gpr.fit(X_train_subset, y_train_subset)


c:\ProgramData\anaconda3\Lib\site-packages\sklearn\gaussian_process\_gpr.py:659: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)


In [ ]:
y_pred = gpr.predict(X_test_scaled)



mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mse)

print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"R² Score: {r2:.4f}")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")

## Lets try another approach
* Reducing the number of features we use

In [ ]:
X= data[['Total Flight Duration(min)','Departure Day','Departure Weekday']]
y = data['Price(NIS)']

NameError: name 'data' is not defined

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)


In [ ]:
# Scale the data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

y_scaler = StandardScaler()
y_train_scaled = y_scaler.fit_transform(y_train.values.reshape(-1, 1)).ravel()
y_test_scaled = y_scaler.transform(y_test.values.reshape(-1, 1)).ravel()


In [ ]:

# Ensure X_train_scaled is a NumPy array
if not isinstance(X_train_scaled, np.ndarray):
    X_train_scaled = X_train_scaled.to_numpy()
if not isinstance(y_train_scaled, np.ndarray):
    y_train_scaled = y_train_scaled.to_numpy()

# Define sample size
sample_size = min(10000, len(X_train_scaled))

# Generate sequential indices to ensure valid bounds
valid_indices = np.arange(len(X_train_scaled))

np.random.seed(101)

# Randomly sample from valid indices
subset_indices = np.random.choice(valid_indices, sample_size, replace=False)

# Select the subset of data
X_train_subset = X_train_scaled[subset_indices]
y_train_subset = y_train_scaled[subset_indices]

# Fit the GPR model
gpr.fit(X_train_subset, y_train_subset)
